In [29]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.embeddings import Embeddings
from openai import OpenAI
from typing import List
from langchain_postgres import PGVector
from rich import print

load_dotenv()

True

In [21]:
folder_path = "D:\\ADK_pregnancy_agent\\crawler"

# Get all .txt files recursively from folder and subfolders
txt_files = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith(".txt"):
            txt_files.append(os.path.join(root, file))

all_pages = []  # To store pages from all files

for txt_file_path in txt_files:
    # print(f"Processing {txt_file_path}...")
    loader = TextLoader(file_path=txt_file_path, encoding="utf-8")

    # Load the documents for this file and append to `all_pages`
    for doc in loader.lazy_load():
        all_pages.append(doc)

# Output all collected pages
print(f"Total pages loaded: {len(all_pages)}")


Total pages loaded: 247

In [22]:
# Splitting document

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800, chunk_overlap=300, separators=["\n"]
)

splits = text_splitter.split_documents(all_pages)
# splits

In [23]:
len(splits)

2945

# Embedding model: BAAI/bge-m3

In [ ]:
class NvidiaOpenAIEmbeddings_BGE_M3(Embeddings):
    def __init__(self, api_key: str, base_url: str, model: str = "baai/bge-m3"):
        self.client = OpenAI(api_key=api_key, base_url=base_url)
        self.model = model

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        response = self.client.embeddings.create(
            input=texts,
            model=self.model,
            encoding_format="float",
            extra_body={"truncate": "NONE"},
        )
        return [data.embedding for data in response.data]

    def embed_query(self, text: str) -> List[float]:
        response = self.client.embeddings.create(
            input=[text],
            model=self.model,
            encoding_format="float",
            extra_body={"truncate": "NONE"},
        )
        return response.data[0].embedding


embedding_engine = NvidiaOpenAIEmbeddings_BGE_M3(
    api_key=os.getenv("NVIDIA_API_KEY"),
    base_url="https://integrate.api.nvidia.com/v1",
    model="baai/bge-m3",
)

In [30]:
SUPABASE_PG_CONN_URL = os.getenv("DB_URI")

vector_store = PGVector(
    embeddings=embedding_engine,
    collection_name="pregnancy_bot",
    connection=SUPABASE_PG_CONN_URL,
    use_jsonb=True,
)
print("PGVector Store is loaded.")

# push embedding to collection
for i in range(0, len(splits), 250):
    chunk = splits[i : i + 250]
    try:
        # Add the chunk to the vector store
        vector_store.add_documents(documents=chunk)
        print(f"Chunk {i // 250} added successfully")
    except Exception as e:
        print(f"Error adding chunk {i // 250}: {e}")
        continue

PGVector Store is loaded.

Chunk 0 added successfully

Chunk 1 added successfully

Chunk 2 added successfully

Chunk 3 added successfully

...

Chunk 7 added successfully

Chunk 8 added successfully

Chunk 9 added successfully

Chunk 10 added successfully

Chunk 11 added successfully

## Vector Retriever

In [31]:
vec_retriever = vector_store.as_retriever(
    search_type="similarity", search_kwargs={"k": 5}
)

In [33]:
docs = vec_retriever.invoke("১১ সপ্তাহে কি খেতে হয়?")
# print(docs)
for doc in docs:
    print(f"Document metadata: {doc.metadata}\n")
    print(f"Document Content: {doc.page_content}")
    print("---------------------------------------------------------------\n")

Document metadata: {'source': 'D:\\ADK_pregnancy_agent\\crawler\\sohay_articles\\গর্ভধারণের ১১তম সপ্তাহ.txt'}

Document Content: গর্ভধারণের ১১তম সপ্তাহ
ক্যাফেইনযুক্ত খাবার ও পানীয় এড়িয়ে চলুন
গর্ভকালীন সময়ে ক্যাফেইন আছে এমন খাবার ও পানীয় এড়িয়ে চলুন। যেমন: চা, কফি, কোক, এনার্জি ড্রিঙ্কস। কারণ ক্যাফেইন গর্ভের শিশুর ওজন কম হওয়াসহ নানানভাবে ক্ষতি
করতে পারে।[১][২][৩]
বসা অথবা শোয়া অবস্থা থেকে হুট করে উঠে দাঁড়াবেন না
অনেকসময় চেয়ারে বসা অথবা বিছানায় শোয়া অবস্থা থেকে হঠাৎ করে দাঁড়িয়ে গেলে ব্রেইনে রক্ত সরবরাহ কমে যেতে পারে। ফলে মাথা ঘুরে পড়ে যাওয়ার সম্ভাবনা বেড়ে যায়। তাই বসা অথবা শুয়ে
থাকা অবস্থা থেকে ওঠার সময়ে ধীরে ধীরে উঠে দাঁড়ানোর চেষ্টা করুন।
আয়রন-ফলিক এসিড ট্যাবলেট সেবন চালিয়ে যান
আপনার শিশু এখন দ্রুত বেড়ে উঠছে। তার চোখ-নাক-কান ও ব্রেইন তৈরি হচ্ছে। এসব যাতে সঠিকভাবে তৈরি হতে পারে, সেজন্য আগের সপ্তাহগুলোর মতো এ সপ্তাহেও অবশ্যই মনে করে
প্রতিদিন আয়রন-ফলিক এসিড ট্যাবলেট সেবন চালিয়ে যাবেন।[৪]

---------------------------------------------------------------

Document metadata: {'source': 'D:\\ADK_pregnancy_agent\\crawler\\firstcry_articles\\আপনার 46 সপ্তাহ বয়সী শিশুর বিকাশ,মাইলস্টোন
এবং যত্ন.txt'}

Document Content: 46 সপ্তাহে আপনার বাচ্চা শক্ত খাবার যেমন ইডলি অথবা প্যান কেক খেতে সক্ষম হয়ে উঠবে।তাকে এগুলো নিজেকেই খেতে দেওয়ার অনুমতি দিন কিন্তু বিষম লাগা
থেকে তাকে প্রতিরক্ষা করতে সে যাতে খাবারগুলো থেকে একবারে একটাই ছোটো টুকরো মুখে ঢোকায় সেটি নিশ্চিত করবেন।এছাড়াও আপনি তাকে আঙ্গুল দিয়ে খাওয়া যায় এরকম খাবার যেমন সবজিকে
লম্বা লম্বা ফিতের মত কেটে সিদ্ধ করে,ফল যেমন কলা,সবেদা,কমলালেবু,মিষ্টি সরবতী লেবু ইত্যাদি দিতে পারেন।তবে আপনার বাচ্চাকে খেতে দেওয়ার আগে ফলগুলিকে খোসা ছাড়িয়ে বীজমুক্ত করা
নিশ্চিত করুন।

...

Document Content: কতটুকু মাছ খাবেন? সপ্তাহে অন্তত ২৮০ গ্রাম মাছ খাওয়ার চেষ্টা করবেন। এর মধ্যে ১৪০ গ্রাম পরিমাণ তৈলাক্ত মাছ খাবেন। যেমন: ইলিশ, পুঁটি, ও
চাপিলা।[৬] খাওয়ার সময়ে ভেজে খাওয়ার পরিবর্তে ভাপে সেদ্ধ করে, কম তেল দিয়ে রান্না করে অথবা সবজি মিশিয়ে তরকারির মতো খেতে পারেন। শিশুকে বুকের দুধ খাওয়ানোর সময়েও এভাবে মাছ
খাওয়া চালিয়ে যাবেন।
বমিভাব ও বমি
এ সময়ে আপনার বমিভাব ও বমি হতে পারে। শরীরে হরমোনের পরিবর্তনের কারণে এই সমস্যাগুলো দেখা দেয় বলে ধারণা করা হয়। বেশিরভাগ ক্ষেত্রেই প্রথম তিন মাস পর বমিভাব ও বমি
কমে আসে।[৭]
আপনার এই সমস্যা হলে কিছু ঘরোয়া উপায়ে সমাধান করার চেষ্টা করতে পারেন। যেমন, আদা খেলে অনেকের বমিভাব ও বমি কমে আসে।[৮][৯], কুসুম গরম পানিতে আদা কুচি দিয়ে খেতে
পারেন বা খানিকটা কাঁচা আদা চিবিয়ে খেতে পারেন।

---------------------------------------------------------------

Document metadata: {'source': 'D:\\ADK_pregnancy_agent\\crawler\\sohay_articles\\গর্ভধারণের চতুর্থ সপ্তাহ.txt'}

Document Content: দুজনে মিলে দিনের খাবারের পরিকল্পনা করুন
গর্ভাবস্থার প্রথম সপ্তাহগুলোতে শিশুর সুস্থ বিকাশের জন্য সুষম খাদ্যাভ্যাস খুব গুরুত্বপূর্ণ। দুজনে মিলে দিনের খাবারগুলো পরিকল্পনা করতে পারেন, যাতে বেশি বেশি শাকসবজি ও ফলমূল
খাওয়া হয় এবং প্রক্রিয়াজাত খাবার কম খাওয়া হয়।
বাজার-সদাইয়ে সচেতন হোন
কী খাবেন সেটা পরিকল্পনা করা হয়ে গেলে সেই অনুযায়ী বাজার করতে সাহায্য করুন। বাজার করার দায়িত্ব আপনার হলে, এসময়ে বেশি বেশি ফল ও শাকসবজি কিনবেন। কয়েক ধরনের ফল ও
শাকসবজি কিনে নিতে পারেন, তাহলে খেতে একঘেয়ে লাগবে না।
স্বাদে ভিন্নতা আসার পাশাপাশি বিভিন্ন ধরনের পুষ্টি উপাদানও পাওয়া যাবে। অন্যদিকে প্রক্রিয়াজাত খাবার কিনে ফেলা সহজ অপশন হলেও সেটা স্বাস্থ্যের জন্য ভালো নয়। তাই এসব খাবার
যতটা সম্ভব এড়িয়ে চলবেন।
ধূমপান ছেড়ে দিন

---------------------------------------------------------------